In [1]:
from keras import optimizers
from keras.layers import Input, Dense, BatchNormalization, Reshape, LeakyReLU, Flatten
from keras.models import Sequential
from keras.models import Model
import numpy as np
from keras.datasets import mnist


class GAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = optimizers.Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        noise_shape = (100,)
        model = Sequential()
        model.add(Dense(128, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        model.summary()
        noise = Input(shape=noise_shape)
        img = model(noise)
        return Model(noise, img)

    def build_discriminator(self):
        img_shape = (self.img_rows, self.img_cols, self.channels)
        model = Sequential()
        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        img = Input(shape=img_shape)
        validity = model(img)
        return Model(img, validity)

    def train(self, epochs, batch_size=128):
        # Load the dataset
        (x_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        x_train = (x_train.astype(np.float32) - 127.5) / 127.5
        x_train = np.expand_dims(x_train, axis=3)
        half_batch = int(batch_size / 2)
        for epoch in range(epochs):
            idx = np.random.randint(0, x_train.shape[0], half_batch)
            imgs = x_train[idx]
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, 100))
            valid_y = np.array([1] * batch_size)
            g_loss = self.combined.train_on_batch(noise, valid_y)
            if epoch % 100 == 0:
              print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))


Using TensorFlow backend.


In [2]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
leak

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.614918, acc.: 46.88%] [G loss: 0.519842]
100 [D loss: 1.353316, acc.: 28.12%] [G loss: 6.601250]
200 [D loss: 0.566558, acc.: 68.75%] [G loss: 4.982168]
300 [D loss: 0.336050, acc.: 96.88%] [G loss: 3.623726]
400 [D loss: 0.478535, acc.: 84.38%] [G loss: 2.679579]
500 [D loss: 0.518039, acc.: 71.88%] [G loss: 1.425866]
600 [D loss: 0.399841, acc.: 87.50%] [G loss: 1.269207]
700 [D loss: 0.532370, acc.: 75.00%] [G loss: 1.182909]
800 [D loss: 0.467541, acc.: 78.12%] [G loss: 1.704172]
900 [D loss: 0.413700, acc.: 84.38%] [G loss: 1.447115]
1000 [D loss: 0.484190, acc.: 78.12%] [G loss: 1.584817]
1100 [D loss: 0.506371, acc.: 84.38%] [G loss: 1.558793]
1200 [D loss: 0.692829, acc.: 62.50%] [G loss: 1.497614]
1300 [D loss: 0.424142, acc.: 84.38%] [G loss: 2.286608]
1400 [D loss: 0.638762, acc.: 65.62%] [G loss: 1.368561]
1500 [D loss: 0.557648, acc.: 84.38%] [G loss: 1.398899]
1600 [D loss: 0.323917, acc.: 87.50%] [G loss: 1.617555]
1700 [D loss: 0.505550, acc.: 81.25%] [G lo